### Import libraries

In [379]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import io
import numpy as np
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Read the Wikipedia URL

In [380]:
#Get the document
wikiURL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(wikiURL).text
wikiText = BeautifulSoup(source, 'lxml')

### Read the codes HTML table into a dataframe

In [381]:
table = wikiText.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

l.pop(0)
td = pd.DataFrame(l, columns=["PostalCode", "Borough","Neighborhood"])
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
pCodes = td[(td.Borough != 'Not assigned') & (td.Borough != '')]
pCodes = pCodes.reset_index(drop ='True')

#Clean values
pCodes['PostalCode']= pCodes['PostalCode'].str.strip()
pCodes['Borough']= pCodes['Borough'].str.strip()
pCodes['Neighborhood']= pCodes['Neighborhood'].str.strip()

### Format the dataframe

In [382]:
# Order the dataset to process
pCodesSort = pCodes.sort_values(by=['PostalCode','Borough','Neighborhood'])

#Create empty DataFrame
pCodesFinal = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

#Loop thru the original DataSet to group data
for index, row in pCodes.iterrows():
    cPCode = row['PostalCode']
    cBorough = row['Borough']
    cNeigh = row['Neighborhood']
    
    #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    if cNeigh=='Not assigned':
        cNeigh = cBorough
    
    if index==0: #First iteration
        pPCode = cPCode
        pBorough = cBorough
        pNeigh = cNeigh
    else:
        if cPCode != pPCode:
            #print( pPCode, pBorough, pNeigh)
            pCodesFinal.loc[index-1] = [pPCode , pBorough , pNeigh] #flush into the new DataSet
            pPCode = cPCode
            pBorough = cBorough
            pNeigh = cNeigh
        else:
            #These two rows will be combined into one row with the neighborhoods separated with a comma 
            pNeigh = pNeigh +", "+ cNeigh

pCodesFinal.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
12,M4B,East York,"Woodbine Gardens, Parkview Hill"
14,M5B,Downtown Toronto,"Ryerson, Garden District"


In [383]:
pCodesFinal.shape

(102, 3)

### Get the GeoData from the file supplied

In [384]:
geoURL = "http://cocl.us/Geospatial_data"
flux = requests.get(geoURL).content
geoDF = pd.read_csv(io.StringIO(flux.decode('utf-8')))
geoDF.columns = ['PostalCode', 'Latitude', 'Longitude'] 
geoDF.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Join the first DataSet with its corresponding Latitude and Longitude

In [385]:
pCodesLatLong = pd.merge( pCodesFinal, geoDF, on='PostalCode', how='left' )
pCodesLatLong.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [386]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(pCodesLatLong['Borough'].unique()),
        pCodesLatLong.shape[0]
    )
)

The dataframe has 11 boroughs and 102 neighborhoods.


In [387]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [418]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(pCodesLatLong['Latitude'], pCodesLatLong['Longitude'], pCodesLatLong['Borough'], pCodesLatLong['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [421]:
# @hidden_cell
CLIENT_ID = 'QYHMAGKNC55K0A4XBZV4MZB5FMULHYZPAYXANCSSXXXGQ1JN' # your Foursquare ID
CLIENT_SECRET = 'OEGOZRL0IZSBUOST3VZSSMEPUQUROEMUMNFFDHW4YYTNZCUC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [390]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [391]:
toronto_venues = getNearbyVenues(names=pCodesLatLong['Neighborhood'],
                                   latitudes=pCodesLatLong['Latitude'],
                                   longitudes=pCodesLatLong['Longitude']
                                  )

In [403]:
print(toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.head()


(2233, 7)
There are 277 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [405]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
#toronto_grouped



(98, 277)


## Cluster Neighborhoods

In [410]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# neighborhoods_venues_sorted.head()


In [411]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print( kmeans.labels_[0:10] )

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = pCodesLatLong

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.fillna(0)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)


[0 0 1 0 0 0 0 0 0 0]


In [419]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Observation

This cluster is almost Parks and Playgrounds, maybe a residential area

In [415]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,1,Fast Food Restaurant,Park,Food & Drink Shop,Ethiopian Restaurant,Event Space
10,North York,1,Pub,Park,Pizza Place,Japanese Restaurant,Women's Store
21,York,1,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy
32,Scarborough,1,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner
35,East York,1,Convenience Store,Park,Coffee Shop,Dumpling Restaurant,Diner
40,North York,1,Bus Stop,Park,Airport,Women's Store,Drugstore
46,North York,1,Park,Grocery Store,Bank,Shopping Mall,Women's Store
49,North York,1,Basketball Court,Park,Bakery,Construction & Landscaping,Dumpling Restaurant
61,Central Toronto,1,Park,Bus Line,Swim School,Women's Store,Donut Shop
64,York,1,Convenience Store,Park,Drugstore,Dim Sum Restaurant,Diner


In [416]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,1,Fast Food Restaurant,Park,Food & Drink Shop,Ethiopian Restaurant,Event Space
10,North York,1,Pub,Park,Pizza Place,Japanese Restaurant,Women's Store
21,York,1,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy
32,Scarborough,1,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner
35,East York,1,Convenience Store,Park,Coffee Shop,Dumpling Restaurant,Diner
40,North York,1,Bus Stop,Park,Airport,Women's Store,Drugstore
46,North York,1,Park,Grocery Store,Bank,Shopping Mall,Women's Store
49,North York,1,Basketball Court,Park,Bakery,Construction & Landscaping,Dumpling Restaurant
61,Central Toronto,1,Park,Bus Line,Swim School,Women's Store,Donut Shop
64,York,1,Convenience Store,Park,Drugstore,Dim Sum Restaurant,Diner
